# Predict Tomorrows Bitcoin Close Price using Classical Machine Learning

In [1]:
from main import run_prediction_pipeline
from helpers.llm_support import get_prompt

print("Forecasting Tomorrows Bitcoin Close Price...")
MODELS_DIRECTORY = r'C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\models\20251109_210801'
df, predictions = run_prediction_pipeline(MODELS_DIRECTORY)
prompt = get_prompt(predictions, df['close'].iloc[-1])
print(predictions)

c:\Users\baile\anaconda3\envs\crypto_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Forecasting Tomorrows Bitcoin Close Price...

Step 1: Ingesting fresh data...
Saving this run's data under: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251111_013413
--- Fetching Quant Data ---


Fetching OHLCV:  95%|█████████▌| 1641600000/1728000000 [00:01<00:00, 939516458.99ms/s]


Quant data saved to: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251111_013413\quant\quant_bitcoin_test_20251111_0134.csv


Device set to use cpu
Batch sentiment: 100%|██████████| 44/44 [04:47<00:00,  6.54s/it]


✅ Google sentiment data saved to: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251111_013413\sentiment\google_news_sentiment_20251111_0146_days_20.csv
--- Fetching Interest Rate Data (Year-by-Year) ---
  -> Fetching: 2024-11-11 to 2025-11-11

🎉 Total Unique Records Fetched: 204
Interest rate data saved to: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251111_013413\interest\interest_rates_test_20251111_0146.csv
--- Starting Data Preprocessing with Temporal Alignment ---
--- Data Preprocessing Complete ---
--- Starting Prediction ---
--- Starting Prediction Pipeline ---
Loading artifacts from: C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\models\20251109_210801
--- Performing Feature Engineering ---

Forecast for the Next Day:
  - Linear Regression: 105686.55
  - Ridge Regression: 105713.39
  - Random Forest: 101058.91
  - XGBoost: 98607.66
{'Linear Regression': 105686.5523796557, 'Ridge Regression': 105713.39198484647, 'Random

## Feed the Classical Machine Learning Predictions into the LLM to have it search the web with context and make a prediction for Tomorrow UTC Close (1600 prior day for PST).

In [2]:
from google import genai
from google.genai import types
from helpers.queries import urls
from helpers.llm_support import add_citations
from dotenv import load_dotenv #comment this out
import os

# Load .env file
load_dotenv()
api_key = os.getenv("gemini_api_key") #replace with your api key
client = genai.Client()

# Build the tools list with each URL as url_context
tools = [
  {"url_context": {}}, #specific user defined urls
  types.Tool(google_search=types.GoogleSearch()) #dynamic search
]

urls = urls() #pulls from dependency

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[prompt + "Look through these and some of your own URLs not listed here: {urls}"],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=-1), #dynamically decides how much it needs to think
        temperature=0.0, # "Highest probability" response. 
        tools=tools
    )
)

text_with_citations = add_citations(response)
print(text_with_citations)

### BTC Price Forecast: November 11, 2025, 16:00 PST (November 12, 2025, 00:00 UTC Close)

*   **Point Estimate:** $105,700.00
*   **Confidence Range:** $104,200.00 to $107,200.00

### 1-Sentence Rationale
> Bitcoin's price is expected to see a modest increase, driven by positive macroeconomic developments and a slight upward bias from the most accurate internal models, despite a prevailing "Fear" sentiment and mixed whale activity.

---

### Detailed Analysis

**1. Model Synthesis:**
*   **Model Consensus:** The Linear Regression ($105,686.55) and Ridge Regression ($105,713.39) models, which exhibit the lowest Mean Absolute Errors (1511.68 and 1523.96 respectively), are tightly clustered around $105.7k. The Random Forest ($101,058.91) and XGBoost ($98,607.66) models, with significantly higher errors, predict lower values.
*   **Internal Signal:** The more accurate models suggest a slight upward movement from yesterday's close of $105,306.6.

**2. Real-Time Market Conditions:**
*   **C